# RFMove Quick Start

In [ ]:
import pybullet
import sys
# You do not need to append "../install/lib" on search path
# if moveit_noros shared library has been installed in to your 
# python site-packages directory.
# sys.path.append("../install/lib")
import moveit_noros as moveit
import pybullet_data
import time
import numpy as np
import threading

## Initialize Pybullet
We use [pybullet](https://github.com/bulletphysics/bullet3) as our test environment. Please refer to [pybullet quickstart guide](https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit) for the usage of pybullet.

In [ ]:
pybullet.connect(pybullet.GUI)
pybullet.setRealTimeSimulation(1)
pybullet.setGravity(0,0,-10)

### Load Robot Into Pybullet

In [ ]:
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
planeId = pybullet.loadURDF("plane.urdf")

pybullet.setAdditionalSearchPath("../resources")
startPos = [0,0,0]
startOrientation = pybullet.getQuaternionFromEuler([0, 0, 0])
robot_id = pybullet.loadURDF("pr2.urdf", startPos, startOrientation, useFixedBase=1)

## Initialize Moveit
### Load Robot Into Moveit
First we need to create a RobotModelLoader through `createRobotModelLoaderFromFile`. RobotModelLoader maintains `RobotModel` and provide shared pointers as views of it. It can also used to generate `RobotState` and `PlanningScene`.

In [ ]:
robot_loader = moveit.createRobotModelLoaderFromFile("../resources/pr2.urdf", "../resources/pr2_config/pr2.srdf")

Then we need to load several configurations into RobotModelLoader including
- JointLimits: Configuration for robot limitations such as position and velocity bounds. It can be loader through `RobotModelLoader.loadJointLimits(JointLimitsLoader)`
- Kinematics configuration: Configuration for kinematics solvers. It can be loaded through `RobotModelLoader.loadKinematicsSolvers(KinematicsLoader)`
- Planner configuration: Configuration for motion planner. PlannerConfiguration works as a parameter when create planner.

In [ ]:
joint_limits_loader = moveit.createJointLimitsLoaderFromFile("../resources/pr2_config/joint_limits.yaml")
robot_loader.loadJointLimits(joint_limits_loader)

kinematic_solver_loader = moveit.createKinematicsLoaderFromFile("../resources/pr2_config/kinematics.yaml")
robot_loader.loadKinematicsSolvers(kinematic_solver_loader)

planner_config = moveit.createPlannerConfigurationFromFile("../resources/pr2_config/ompl_planning.yaml")

### Create Planner and PlanningScene
Planner is the motion planning solver. PlanningScene matains the robot and world info.

In [ ]:
robot_model = robot_loader.getModel()
planner = moveit.PlannerManager(robot_model, planner_config)
planning_scene = robot_loader.newPlanningScene()

### Create Pybullet Hardware Accessor
Robot controller is implemented within our moveit_noros shared library. The controller can be used to control different hardwares as long as the hardware has implemented the common `HardwareInterface`.

When it comes to pybullet, we also treat it as a hardware and implement `PybulletHardware` inherited from `HardwareInterface`. However, the implementation is very intuitively which accesses the pybullet python methods directly.

In order to create a PybulletHardware, we need to provide pybullet python module directly together with the bullet3 bodyUniqueId of robot, which is the return value of `pybullet.loadURDF`.

In [ ]:
bullet_hardware = moveit.PybulletHardware(pybullet, robot_id)

### Create PlanningSceneHelper
PlanningSceneHelper has the following functionality:
- Synchronize robot state between moveit planning scene and simulated / real robot hardware.
- Synchronize world information between moveit planning scene and simulated / real world.

In [ ]:
scene_helper = moveit.PlanningSceneHelper(bullet_hardware, planning_scene)

PlanningSceneHelper provides two different robot state synchronization method
- `sync()`: Change the robot state within moveit PlanningScene the same as pybullet hardware.
- `resync()`: Change the robot state (joint position only) within pybullet the same as PlanningScene current robot state.

The first thing we need to do after creation of PlanningSceneHelper is call `PlanningSceneHelper.reset()` which does two important setting
- Set the current robot state within PlanningScene as default state.
- Set all joint positions within pybullet the same as PlanningScene, call `resync()` in other words.

In [ ]:
scene_helper.reset()

However, you may find out that the robot in pybullet GUI is non draggable after the calling of `PlanningSceneHelper.reset()` or `resync()`. That is because giant force is applied on every robot joint. You can call `PybulletHardware.free()` to make it draggable again.

If you still can not drag the robot link through your mouse, try to press `ALT` or `CTRL`.

In [ ]:
bullet_hardware.free()

`free()` can also have one parameter which specify the force value applied on each joint. You can try to give it a small value (default is 200).

In [ ]:
bullet_hardware.free(1)

In [ ]:
scene_helper.reset()

## Solve Inverse Kinematics
Inverse kinematics means to calculate the position of each joint according to the position of tip link. Solving IK does not need any planner or planning scene. It can be done through `RobotState.setFromIK()` directly.

In order to test IK solving, we need to specify the JointModelGroup we calculate for. JointModelGroup is the basic unit to do kinematics calculation and motion planning. It is defined within the moveit `srdf` file.

In [ ]:
PLANNING_GROUP = "left_arm"
joint_model_group = robot_model.getJointModelGroup(PLANNING_GROUP)

base_link = joint_model_group.base_link
tip_link = joint_model_group.tip_link
print("joint model group:\t%s" % joint_model_group)
print("base link:\t %s" % base_link)
print("tip link:\t %s" % tip_link)

Any robot state can be used to compute IK. Here we use two different robot state:
- One random state, which generates valid random robot position.
- One control state. We use the tip link position generated by random state as input, set the control state by IK solving.

In [ ]:
random_state = robot_loader.newRobotState()
control_state = robot_loader.newRobotState()
random_state.setToDefaultValues()
control_state.setToDefaultValues()

We first set random_state to a valid random position and fetch its tip link transform. However, here we only want to solve IK for `left_arm`, so we use joint_model_group as one parameter. `RobotState.setToRandomPositions()` can also take no parameter which means set all joints to a random position.

**Note:** Transformation here is represented through Eigen Affine Matrix.

In [ ]:
random_state.setToRandomPositions(joint_model_group)
tip_transform = random_state.getGlobalLinkTransform(tip_link)
print(tip_transform.__repr__())

Now we set control state through IK solving.

In [ ]:
success = control_state.setFromIK(joint_model_group, tip_transform)
print("IK FOUND" if success else "CAN NOT FIND IK")

We can visualize the IK solution by control pybullet model move to control_state. Here we use `JointModelGroup.getJointModelNames()` to fetch the name of all joints. And use `PybulletHardware.getJointIndex(joint_name)` to fetch the joint index of specific joint within pybullet.

In [ ]:
for joint_name in joint_model_group.getJointModelNames():
    joint_index = bullet_hardware.getJointIndex(joint_name)
    if joint_index > 0:
        pybullet.setJointMotorControl2(robot_id, 
                                       joint_index, 
                                       pybullet.POSITION_CONTROL, 
                                       targetPosition = control_state.getJointPosition(joint_name),
                                       maxVelocity = 3)
    else:
        print("Can not find joint " + joint_name)

Here we use a small thread to repeat the above process.

In [ ]:
class randomIK(threading.Thread):
    def __init__(self):
        self.stop = False
        threading.Thread.__init__(self)
    def run(self):
        while True:
            random_state.setToRandomPositions(joint_model_group)
            tip_transform = random_state.getGlobalLinkTransform(tip_link)
            if(control_state.setFromIK(joint_model_group, tip_transform)):
                for joint_name in joint_model_group.getJointModelNames():
                    joint_index = bullet_hardware.getJointIndex(joint_name)
                    if joint_index > 0:
                        pybullet.setJointMotorControl2(robot_id, 
                                                       joint_index, 
                                                       pybullet.POSITION_CONTROL, 
                                                       targetPosition = control_state.getJointPosition(joint_name),
                                                       maxVelocity = 3)
                    else:
                        print("Can not find joint " + joint_name)
            if self.stop:
                break
            time.sleep(2)
        print("Thread end")

In [ ]:
randomIK_thread = randomIK()
randomIK_thread.start()

In [ ]:
randomIK_thread.stop = True
scene_helper.reset()

## Pose Goal Motion Plan
The most motion plan task is move the robot to a specific pose. Here we use planner to set the tip link of joint_model_goup to specific pose. This may sounds same as inverse kinematics. But we can get a whole moveing trajectory with the help of planner.

Remember that we have created our planner and planning scene before.

In [ ]:
print(planner.__repr__())
print(planning_scene.__repr__())
print(scene_helper.__repr__())

We still use random_state to generate a random tip_link transform.

In [ ]:
random_state.setToRandomPositions(joint_model_group)
tip_transform = random_state.getGlobalLinkTransform(tip_link)
print(tip_transform.__repr__())

In order to do a motion plan, we need to construct a MotionPlanRequest. The definition of MotionPlanRequest is from [ros-planning/moveit_msgs/msg/MotionPlanRequest.msg](https://github.com/ros-planning/moveit_msgs/blob/kinetic-devel/msg/MotionPlanRequest.msg).

We first create a stamped pose for tip link. `PoseStamped` need to specify the corresponding frame. All transforms fetched directly from `RobotState` is according to robot root frame. Here we want to use the joint model group base frame instead of the robot root frame. In that case, we need to compute the relative transform of tip_link according to the group base frame.

`RobotState.linkRelativeTransform()` can calculate the relative transform of link according to specific frame.

When creating PoseStamped, the rotation should be in quaternion form.

In [ ]:
base_frame = joint_model_group.base_frame
relative_transform = random_state.linkRelativeTransform(tip_link, base_frame)
print(relative_transform)
pose = moveit.PoseStamped(base_frame, relative_transform.translation, relative_transform.quaternion.value)

Create MotionPlanRequest and add a pose goal.

In [ ]:
req = moveit.MotionPlanRequest(PLANNING_GROUP)
pose_goal = moveit.constructGoalConstraints(tip_link, pose, 0.01, 0.01)
req.addGoal(pose_goal)

Create planning context for this request.

In [ ]:
context = planner.getPlanningContext(planning_scene, req)

Solve motion plan task.

In [ ]:
response = context.solve()

In [ ]:
response.trajectory

In [ ]:
bullet_controller = moveit.Controller(bullet_hardware, 0.01)

In [ ]:
scene_helper.sync()

In [ ]:
bullet_hardware.free()

In [ ]:
bullet_hardware.stayCurrent()

In [ ]:
req

In [ ]:
relative_transform

In [ ]:
base_transform.inverse * link_transform

In [ ]:
response = context.solve()


In [ ]:
response.trajectory

In [ ]:
for i in range(len(spline.tip_transforms)-1):
    p.addUserDebugLine(spline.tip_transforms[i].translation, spline.tip_transforms[i+1].translation)

In [ ]:
p.addUserDebugLine(np.array([0.629203, 0.142431, 0.705641]), np.array([0.562863, 0.367344, 1.18433]), [1,0,0], 5)

In [ ]:
transform

In [ ]:
transform.quaternion

In [ ]:
transform.quaternion.value

In [ ]:
transform.quaternion.matrix

In [ ]:
base_frame

In [ ]:
transform.quaternion[0]

In [ ]:
robot_model.getJointModelGroup

In [ ]:
moveit.createKinematicsLoaderFromFile